In [66]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier


In [67]:
# Load the data
file_path = Path('Resources/teds_2019_cleaned.csv')
teds_cleaned_df = pd.read_csv(file_path)

teds_cleaned_df.head()

,AGE,RACE,GENDER,EDUC,MARSTAT,EMPLOY,EMPLOY.1,LIVARAG,SERVICES,SUB1,...,FRSTUSE1,ALCFLG,COKEFLG,MARFLG,MTHAMFLG,OPSYNFLG,HERFLG,FREQ1,REASON,SUCCESSFUL
0,11,8,2,3,3,4,4,3,7,2,...,7,1,0,0,0,0,0,1,3,0
1,5,5,2,3,1,1,1,3,7,2,...,2,1,0,0,0,0,0,1,1,1
2,6,7,2,3,1,1,1,3,7,2,...,2,1,0,0,0,0,0,2,3,0
3,5,8,2,3,1,4,4,2,7,2,...,3,1,0,1,0,0,0,1,3,0
4,5,8,2,2,1,4,4,3,7,2,...,3,1,0,1,0,0,0,2,3,0


# Create intitial neural network model

In [79]:
# Split our preprocessed data into our features and target arrays

y = teds_cleaned_df["SUCCESSFUL"].values
X = teds_cleaned_df.drop(["REASON","SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

C:\Users\syode\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [80]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [81]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 8)                 168       
                                                                 
 dense_25 (Dense)            (None, 5)                 45        
                                                                 
 dense_26 (Dense)            (None, 1)                 6         
                                                                 
Total params: 219
Trainable params: 219
Non-trainable params: 0
_________________________________________________________________


In [82]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [83]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=10)

Epoch 1/10
15659/15659 [==============================] - 18s 1ms/step - loss: 0.5411 - accuracy: 0.7432
Epoch 2/10
15659/15659 [==============================] - 18s 1ms/step - loss: 0.5354 - accuracy: 0.7462
Epoch 3/10
15659/15659 [==============================] - 18s 1ms/step - loss: 0.5344 - accuracy: 0.7467
Epoch 4/10
15659/15659 [==============================] - 18s 1ms/step - loss: 0.5334 - accuracy: 0.7469
Epoch 5/10
15659/15659 [==============================] - 18s 1ms/step - loss: 0.5328 - accuracy: 0.7468
Epoch 6/10
15659/15659 [==============================] - 18s 1ms/step - loss: 0.5324 - accuracy: 0.7471
Epoch 7/10
15659/15659 [==============================] - 17s 1ms/step - loss: 0.5323 - accuracy: 0.7469
Epoch 8/10
15659/15659 [==============================] - 17s 1ms/step - loss: 0.5321 - accuracy: 0.7471
Epoch 9/10
15659/15659 [==============================] - 16s 1ms/step - loss: 0.5319 - accuracy: 0.7467
Epoch 10/10
15659/15659 [==============================

In [84]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5220/5220 - 4s - loss: 0.5318 - accuracy: 0.7477 - 4s/epoch - 739us/step
Loss: 0.5318129062652588, Accuracy: 0.7477323412895203


# Create initial Random Forest Classifier Model

In [85]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")


 Random forest predictive accuracy: 0.727
